In [147]:
import numpy as np

filename = "cleveland.data"

to_int = np.vectorize(float)

with open(filename, 'r', encoding='ascii') as f:
    values = [word for line in f for word in line.split()]
    data_processed = np.reshape(np.array(values), (-1, 76))

flatten = np.vectorize(lambda x: 1 if x != 0 else 0)
Y = flatten(to_int(data_processed[:, 58 - 1]))
    
# exclude the predictor
X = to_int(np.delete(data_processed, 1)) 
    
# exclude columns for which all non-diseased people have the same value
garbage_columns = [np.unique(X[Y!=1,attr]).size == 1 | attr >= 68 for attr in range(74)]
columns_to_exclude = np.where(np.array(garbage_columns))[0]

# diabetic?
d = data_processed[:, 17 - 1]

ValueError: could not convert string to float: 'name'

In [148]:
Y

array([0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0])

Exploratory data analysis using ANOVA:

In [149]:
import scipy.stats as stats

diabetic = X[:, 17 - 1] == 1
stats.f_oneway(Y[diabetic], Y[np.logical_not(diabetic)])

F_onewayResult(statistic=2.7849569187612762, pvalue=0.096270975783822321)

ANOVA indicates a p-value of 0.09, a little promising. Now let's try basic logistic regression:

In [150]:
from sklearn import linear_model
from sklearn.model_selection import KFold


kf.get_n_splits(X)
print(kf)

def kfold_validate (X_sub, k):
    kf = KFold(n_splits = k)
    sum = 0
    for train, test in kf.split(X_sub):
        reg = linear_model.LogisticRegression()
        reg.fit(X_sub[train], Y[train])
        sum += (reg.score(X_sub[test], Y[test]))
    return sum / k
        
def filter_select(X, Y, num_features):
    # ignore the bad columns
    avail_features = list(range(0, 74))
    for garbage_col in columns_to_exclude:
        avail_features.remove(garbage_col)
        
    included_features = [] #[17 - 1] # diabetes    
    for i in range(num_features):
        best_feature = -1
        best_score = -1
        for feature in avail_features:
            score = kfold_validate(X[:,included_features + [feature]], 4)
            if score > best_score:
                best_score = score
                best_feature = feature
        included_features += [best_feature]
        avail_features.remove(best_feature)
        print(included_features, best_score)
    return included_features
            
filter_select(X, Y, 10)
#for i in range(76):
#    reg = linear_model.LogisticRegression()
#    reg.fit(X, Y)
#    reg.coef_

KFold(n_splits=4, random_state=None, shuffle=False)
[50] 0.758400402414
[50, 70] 0.776257545272
[50, 70, 8] 0.793963782696
[50, 70, 8, 31] 0.815291750503
[50, 70, 8, 31, 43] 0.829527162978
[50, 70, 8, 31, 43, 18] 0.84009054326
[50, 70, 8, 31, 43, 18, 30] 0.850754527163
[50, 70, 8, 31, 43, 18, 30, 40] 0.854325955734
[50, 70, 8, 31, 43, 18, 30, 40, 38] 0.857847082495
[50, 70, 8, 31, 43, 18, 30, 40, 38, 22] 0.857847082495


[50, 70, 8, 31, 43, 18, 30, 40, 38, 22]

In [146]:
np.mean(X[Y==1, 50]),np.mean(X[Y==0, 50])

(5.6879999999999997, 3.7006369426751591)

In [ ]:
from scipy.stats import linregress

